# 插补用户用电量数据缺失值


1.读取missing_data.csv数据

In [52]:
#导入pandas模块
import pandas as pd
pd.*?

In [53]:

#读取missing_data数据,格式为xls故用read_excel
missing_data = pd.read_excel("./five/missing_data.xls")

2.查询缺失值位置

In [54]:
i = 0
for columname in missing_data.columns:
    if(missing_data[columname].count() != len(missing_data)):
        loc = missing_data[columname][missing_data[columname].isnull().values==True].index.tolist()
        print('列："',i+1,'", 第{}行位置有缺失值'.format(loc))
        i = i+1
        

列：" 1 ", 第[4, 11]行位置有缺失值
列：" 2 ", 第[2, 9, 18, 19]行位置有缺失值
列：" 3 ", 第[3, 6, 9, 15]行位置有缺失值


3.拉格朗日插值法补空值

In [55]:
import numpy as np#导入模块
from scipy.interpolate import lagrange#导入函数

data=pd.read_excel('five/missing_data.xls')
#自定义列向量插值函数
def ploy(s,n,k=6):
    y=s[list(range(n-k,n))+list(range(n+1,n+1+k))]#取数
    y=y[y.notnull()]
    return lagrange(y.index,list(y))(n)#拉格朗日补值
for i in data.columns:#遍历列
    for j in range(len(data)):
        if(data[i].isnull())[j]:
            data[i][j]=ploy(data[i],j)#调用函数
data.to_excel('five/Completedata.xls')


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:841: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


4.查看是否还有缺失值

In [56]:
test_data = pd.read_excel('five/Completedata.xls')
test_data

,235.8333,324.0343,478.3231
0,236.270800,325.637900,515.456400
1,238.052100,328.089700,517.090900
2,235.906300,114.852129,514.890000
3,236.760400,268.832400,486.475945
4,236.034254,404.048000,486.091200
5,237.416700,391.265200,516.233000
6,238.656300,380.824100,490.735041
7,237.604200,388.023000,435.350800
8,238.031300,206.434900,487.675000
9,235.072900,233.438198,609.621137


## 合并线损、用电量趋势与线路告警数据

1.读取ele_loss.csv和alarm.csv表

In [57]:
#加载表的数据
ele_loss = pd.read_csv("five/ele_loss.csv",encoding='gbk')
alarm = pd.read_csv("five/alarm.csv",encoding="gbk")

2.查看两表形状

In [58]:
print("ele_loss.csv形状：",ele_loss.shape)
print("alarm.csv形状：",alarm.shape)

ele_loss.csv形状： (49, 4)
alarm.csv形状： (25, 3)


3.通过id和data两个键作为主键进行内连接

In [59]:
res = pd.merge(ele_loss,alarm,how="inner",on=['ID','date'],sort=0)

In [60]:
#查看数据
res

,ID,date,ele,loss,alarm
0,21261001,2010/9/3,858.0,0.151048,电流不平衡
1,21261001,2010/9/6,883.5,0.162778,C相电流过负荷
2,21261001,2010/9/16,915.5,0.162028,C相电流过负荷
3,21261001,2010/9/16,915.5,0.162028,C相电流过负荷
4,21261001,2010/9/17,961.0,0.166650,A相电流过负荷
5,21261001,2010/9/17,961.0,0.166650,电流不平衡


## 标准化建模专家样本数据

1.读取model.csv数据

In [61]:
model = pd.read_excel('five/model.xls')

In [62]:
model

,电量趋势下降指标,线损指标,告警类指标,是否窃漏电
0,4,1,1,1
1,4,0,4,1
2,2,1,1,1
3,9,0,0,0
4,3,1,0,0
5,2,0,0,0
6,5,0,2,1
7,3,1,3,1
8,3,0,0,0
9,4,1,0,0


2.定义标准差标准化函数

In [63]:
def StandardScaler(data):
    data=(data-data.mean())/data.std()    
    return data 

3.使用函数分别对3列数据进行标准化
4.查看标准化的数据

In [64]:
data1 = StandardScaler(model['电量趋势下降指标'])
data1

0      0.612777
1      0.612777
2     -0.343352
3      3.003099
4      0.134712
5     -0.343352
6      1.090841
7      0.134712
8      0.134712
9      0.612777
10     3.481163
11     3.481163
12    -0.343352
13     0.612777
14     0.134712
15    -1.299481
16     3.003099
17    -1.299481
18     2.525034
19    -0.343352
20     0.134712
21     2.046970
22     1.568906
23     0.612777
24     2.046970
25    -0.343352
26     1.090841
27    -0.821417
28     1.090841
29    -0.821417
         ...   
261    1.090841
262   -0.343352
263    1.090841
264   -0.343352
265   -0.343352
266    0.134712
267    1.090841
268    0.612777
269   -0.821417
270    0.612777
271   -0.343352
272   -0.821417
273   -0.821417
274   -0.343352
275    0.134712
276   -1.299481
277   -1.299481
278   -1.299481
279   -0.821417
280   -0.343352
281    0.134712
282    0.612777
283   -0.343352
284    0.134712
285    0.134712
286    0.612777
287   -0.821417
288    1.090841
289   -0.343352
290    0.612777
Name: 电量趋势下降指标, Length: 

In [65]:
data2 = StandardScaler(model['线损指标'])
data2

0      0.974549
1     -1.022590
2      0.974549
3     -1.022590
4      0.974549
5     -1.022590
6     -1.022590
7      0.974549
8     -1.022590
9      0.974549
10     0.974549
11     0.974549
12    -1.022590
13    -1.022590
14    -1.022590
15    -1.022590
16    -1.022590
17    -1.022590
18     0.974549
19    -1.022590
20    -1.022590
21    -1.022590
22    -1.022590
23     0.974549
24    -1.022590
25     0.974549
26     0.974549
27     0.974549
28     0.974549
29    -1.022590
         ...   
261    0.974549
262   -1.022590
263   -1.022590
264   -1.022590
265   -1.022590
266    0.974549
267    0.974549
268    0.974549
269   -1.022590
270   -1.022590
271    0.974549
272    0.974549
273   -1.022590
274    0.974549
275   -1.022590
276    0.974549
277    0.974549
278    0.974549
279   -1.022590
280   -1.022590
281   -1.022590
282   -1.022590
283    0.974549
284    0.974549
285   -1.022590
286    0.974549
287   -1.022590
288    0.974549
289    0.974549
290    0.974549
Name: 线损指标, Length: 291,

In [66]:
data3 = StandardScaler(model['告警类指标'])
data3

0     -0.161216
1      2.653616
2     -0.161216
3     -1.099494
4     -1.099494
5     -1.099494
6      0.777061
7      1.715339
8     -1.099494
9     -1.099494
10     0.777061
11     1.715339
12     1.715339
13     0.777061
14    -1.099494
15     1.715339
16     1.715339
17     0.777061
18     2.653616
19     2.653616
20    -0.161216
21    -1.099494
22    -1.099494
23     0.777061
24    -1.099494
25    -1.099494
26    -1.099494
27     2.653616
28    -0.161216
29     0.777061
         ...   
261    0.777061
262    0.777061
263   -1.099494
264    0.777061
265   -1.099494
266    0.777061
267   -1.099494
268    0.777061
269   -0.161216
270    0.777061
271   -0.161216
272    0.777061
273   -1.099494
274   -0.161216
275   -0.161216
276   -0.161216
277   -1.099494
278   -0.161216
279   -0.161216
280    0.777061
281    0.777061
282   -1.099494
283   -1.099494
284    0.777061
285   -0.161216
286    0.777061
287    0.777061
288    0.777061
289   -1.099494
290   -1.099494
Name: 告警类指标, Length: 291